In [46]:
import numpy as np
import pandas as pd

from urllib.parse import urljoin
from urllib.request import urlopen
from urllib.error import HTTPError
from bs4 import BeautifulSoup, SoupStrainer
from os.path import basename, dirname
from decimal import Decimal

pd.set_option('display.max_rows', 500)

In [2]:
def scrape_url(url): #, url_list=[], url_root=''):

    result=[]
    url_list=[]
    url_root=dirname(url)
    
    def scrape_url(url, url_list, url_root):
        
        url_base=dirname(url)
        url_list.append(url)
        
        if not url_root in url_base:
            return
    
        try:
            content=urlopen(url).read()
        except HTTPError as e:
            #print(e.code)
            #print(e.msg)
            return

        print('Processing: ', url)
        try:
            data=pd.read_html(content)
            print(str(len(data))+" tables were found")
            result.append(data[0])
        except:
            print("No tables were found")
            
        soup=BeautifulSoup(content, 'html.parser')
    
        for link in soup.find_all('a'):
            url=link.get('href')
            if not 'http' in url:
                url = urljoin(url_base+'/index.html', url, False)
        
            if not (url in url_list):
                scrape_url(url, url_list, url_base)
                
    scrape_url(url, url_list, url_root)

    print("Number of processed tables: "+str(len(result)))
    #result=pd.concat(result)
    #print("Number of columns: "+str(len(result.columns)))
    #print("Number of elements: "+str(len(result.index)))
    
    return result

In [4]:
conicyt_url="http://www.conicyt.cl/transparencia/transferencias/transferencias_historico.html"
conicyt_url="http://www.conicyt.cl/transparencia/transferencias/2010/transferencias_meses_2010.html"
#conicyt_url="http://www.conicyt.cl/transparencia/transferencias/2015/01-enero/transferencias.html"

data=scrape_url(conicyt_url)

Processing:  http://www.conicyt.cl/transparencia/transferencias/2010/transferencias_meses_2010.html
No tables were found
Processing:  http://www.conicyt.cl/transparencia/transferencias/2010/enero/transferencias_enero.html
1 tables were found
Processing:  http://www.conicyt.cl/transparencia/transferencias/2010/enero/transferencias-2.html
1 tables were found
Processing:  http://www.conicyt.cl/transparencia/transferencias/2010/enero/transferencias-3.html
1 tables were found
Processing:  http://www.conicyt.cl/transparencia/transferencias/2010/enero/transferencias-4.html
1 tables were found
Processing:  http://www.conicyt.cl/transparencia/transferencias/2010/enero/transferencias-5.html
1 tables were found
Processing:  http://www.conicyt.cl/transparencia/transferencias/2010/enero/transferencias-6.html
1 tables were found
Processing:  http://www.conicyt.cl/transparencia/transferencias/2010/enero/transferencias-7.html
1 tables were found
Processing:  http://www.conicyt.cl/transparencia/transfe

### Chequeamos si el nombre de las columnas es consistente para todos los Dataframe

In [54]:
data[0].head()

,Mes transferencia,Fecha transferencia,Finalidad transferencia,Respaldo transferencia,Unidad monetaria,Monto transferencia,RUT,Razón social,Apellido paterno,Apellido materno,Nombres,Región de la organización,Año transferencia
0,Enero,27-01-2010,PROYECTO 31090001 INV. JORGE LUIS ALFARO,Resolución N° 126 del 28/01/2004 Aprueba Conve...,Pesos,7912254.0,81698900-0,PONTIFICIA UNIVERSIDAD CATOLICA DE CHILE,NaN,NaN,NaN,RM,2010
1,Enero,27-01-2010,PROYECTO 31090002 INV. LUIS FELIPE BARRIENTOS,Resolución N° 126 del 28/01/2004 Aprueba Conve...,Pesos,9494705.0,81698900-0,PONTIFICIA UNIVERSIDAD CATOLICA DE CHILE,NaN,NaN,NaN,RM,2010
2,Enero,27-01-2010,PROYECTO 31090009 INV. GASPAR GALAZ,Resolución N° 126 del 28/01/2004 Aprueba Conve...,Pesos,3164902.0,81698900-0,PONTIFICIA UNIVERSIDAD CATOLICA DE CHILE,NaN,NaN,NaN,RM,2010
3,Enero,27-01-2010,PROYECTO 31090015 INV. ANDRES JORDAN,Resolución N° 126 del 28/01/2004 Aprueba Conve...,Pesos,18461926.0,81698900-0,PONTIFICIA UNIVERSIDAD CATOLICA DE CHILE,NaN,NaN,NaN,RM,2010
4,Enero,27-01-2010,PROYECTO 31090020 INV. VERONICA MOTTA,Resolución N° 126 del 28/01/2004 Aprueba Conve...,Pesos,12803000.0,65064060-8,SOCIEDAD CHILENA DE ASTRONOMIA,NaN,NaN,NaN,RM,2010


In [5]:
conicyt=pd.concat(data)
conicyt.columns

Index([' Apellido materno, si el receptor es persona natural\n ',
       ' Apellido paterno, si el receptor es persona natural\n ',
       ' Fecha de la transferencia(dd/mm/aaaa)\n ',
       ' Mes de la transferencia\n ', ' Monto\n ',
       ' Nombres si el receptor es persona natural\n ',
       ' RUT (si es persona jurídica)\n ',
       ' Razón social de la persona jurídica\n ',
       ' Región de la organización que recibe el aporte\n ',
       ' Respaldo jurídico de la transferencia\n ', ' Unidad monetaria\n ',
       ' Utilidad o finalidad de la transferencia\n ',
       'Apellido materno, si el receptor es persona natural',
       'Apellido materno, si receptor es persona natural',
       'Apellido paterno, si el receptor es persona natural',
       'Apellido paterno, si receptor es persona natural',
       'Fecha de la transferencia(dd/mm/aaaa)', 'Imputación presupuestaria',
       'Mes de la transferencia', 'Monto',
       'Nombres si el receptor es persona natural',
       'No

In [59]:
for i in range(len(data)):
    data[i].rename(columns={'Apellido paterno, si el receptor es persona natural':'Apellido paterno',
                            'Apellido paterno, si receptor es persona natural':'Apellido paterno',
                            ' Apellido paterno, si el receptor es persona natural\n ':'Apellido paterno',
                            'Apellido materno, si el receptor es persona natural':'Apellido materno',
                            'Apellido materno, si receptor es persona natural':'Apellido materno',
                            ' Apellido materno, si el receptor es persona natural\n ':'Apellido materno',
                            'Nombres si el receptor es persona natural':'Nombres',
                            'Nombres, si receptor es persona natural':'Nombres',
                            ' Nombres si el receptor es persona natural\n ':'Nombres',
                            'Fecha de la transferencia(dd/mm/aaaa)':'Fecha transferencia',
                            ' Fecha de la transferencia(dd/mm/aaaa)\n ':'Fecha transferencia',
                            'Mes de la transferencia':'Mes transferencia',
                            ' Mes de la transferencia\n ':'Mes transferencia',
                            'RUT (si es persona jurídica)':'RUT',
                            ' RUT (si es persona jurídica)\n ':'RUT',
                            'RUT, si receptor es persona jurídica':'RUT',
                            'Razón social de la persona jurídica':'Razón social',
                            'Razón social de la persona jurídica, si corresponde':'Razón social',
                            ' Razón social de la persona jurídica\n ':'Razón social',
                            'Respaldo jurídico de la transferencia':'Respaldo transferencia',
                            ' Respaldo jurídico de la transferencia\n ':'Respaldo transferencia',
                            'Utilidad o finalidad de la transferencia':'Finalidad transferencia',
                            ' Utilidad o finalidad de la transferencia\n ':'Finalidad transferencia',
                            'Región de la organización que recibe el aporte':'Región de la organización',
                            ' Región de la organización que recibe el aporte\n ':'Región de la organización',
                            'Monto':'Monto transferencia',
                            ' Monto\n ':'Monto transferencia',
                            ' Unidad monetaria\n ':'Unidad monetaria'},
                   inplace=True)
    data[i]['Fecha transferencia'] = pd.to_datetime(data[i]['Fecha transferencia'])
    data[i]['Año transferencia'] = data[i]['Fecha transferencia'].dt.year #data[i].apply( lambda x: pd.to_datetime(x['Fecha transferencia']).year, axis=1)
    data[i]['Monto transferencia'] = data[i].apply( lambda x: float(x['Monto transferencia'].replace('.','')) if isinstance(x['Monto transferencia'], str) else float(x['Monto transferencia']), axis=1)

In [83]:
conicyt=pd.concat(data).reindex(columns=['Fecha transferencia','Año transferencia','Mes transferencia','Finalidad transferencia','Respaldo transferencia','Imputación presupuestaria','Monto transferencia','Unidad monetaria','RUT','Razón social','Apellido paterno','Apellido materno','Nombres','Región de la organización'])
conicyt=conicyt.sort_values('Fecha transferencia').reset_index(drop=True)
conicyt.head()

,Fecha transferencia,Año transferencia,Mes transferencia,Finalidad transferencia,Respaldo transferencia,Imputación presupuestaria,Monto transferencia,Unidad monetaria,RUT,Razón social,Apellido paterno,Apellido materno,Nombres,Región de la organización
0,2010-01-14,2010,Enero,Doctorado 2 2009-2 72100349,"DS. N° 664 del 29 de Diciembre de 2008, del Mi...",NaN,3705700.0,Pesos,NaN,NaN,BLANCO,JARA,JOSE,NaN
1,2010-01-14,2010,Enero,Doctorado 2 2009-2 72101165,"DS. N° 664 del 29 de Diciembre de 2008, del Mi...",NaN,3589397.0,Pesos,NaN,NaN,CASTAÑEDA,PEÑA,ROSELYN,NaN
2,2010-01-15,2010,Enero,DOCT 1 2009 72091017,"DS. N° 664 del 29 de Diciembre de 2008, del Mi...",NaN,11709209.0,Pesos,NaN,NaN,GARCIA HUIDOBRO,BECERRA,CRISTOBAL,NaN
3,2010-01-15,2010,Enero,BECA EXTRA. ASIG. MANUT. DOC EXT 13075559,Decreto supremo 29 del 29/12/1988,NaN,935221.0,Pesos,NaN,NaN,LLADSER,CALDERA,ALVARO FERNANDA,NaN
4,2010-01-15,2010,Enero,BECA EXTRA. ASIG. MANUT. DOC EXT 15651030,Decreto supremo 29 del 29/12/1988,NaN,625124.0,Pesos,NaN,NaN,NUÑEZ,CATALAN,MARCIA ANDREA,NaN


In [90]:
gv=conicyt['Finalidad transferencia'].str.contains('DOCTORADO 2010')
conicyt_doctorado=conicyt[gv].reset_index(drop=True)
conicyt_doctorado['Finalidad transferencia'].unique()

array(['BECAS NAC. ASIG. MENSU DOCTORADO 2010 14631527',
       'BECAS NAC. ASIG. MENSU DOCTORADO 2010 15177658',
       'BECAS NAC. ASIG. MENSU DOCTORADO 2010 16094162',
       'BECAS NAC. ASIG. MENSU DOCTORADO 2010 8376250',
       'BECAS NAC. ASIG. MENSU DOCTORADO 2010 7220035',
       'BECAS NAC. ASIG. MENSU DOCTORADO 2010 14478014',
       'BECAS NAC. ASIG. MENSU DOCTORADO 2010 16439839',
       'BECAS NAC. ASIG. MENSU DOCTORADO 2010 16155888',
       'BECAS NAC. ASIG. MENSU DOCTORADO 2010 15853783',
       'BECAS NAC. ASIG. MENSU DOCTORADO 2010 10604815',
       'BECAS NAC. ASIG. MENSU DOCTORADO 2010 15078335',
       'BECAS NAC. ASIG. MENSU DOCTORADO 2010 15015806',
       'BECAS NAC. ASIG. MENSU DOCTORADO 2010 15311880',
       'BECAS NAC. ASIG. MENSU DOCTORADO 2010 13054413',
       'BECAS NAC. ASIG. MENSU DOCTORADO 2010 10678210',
       'BECAS NAC. ASIG. MENSU DOCTORADO 2010 15152626',
       'BECAS NAC. ASIG. MENSU DOCTORADO 2010 15071715',
       'BECAS NAC. ASIG. MENSU DO

### Una revisión de la columna Finalidad transferencia nos permite identificar que la palabra NACIONAL aparece abreviado como "NAC" y "NAC.". También notamos que el número de Folio es el último valor que aparece en el campo

In [95]:
conicyt_doctorado['Finalidad transferencia']=conicyt_doctorado.apply(lambda x: x['Finalidad transferencia'].replace('NAC.','NACIONAL').replace('NAC ','NACIONAL '), axis=1)
conicyt_doctorado['Folio']=conicyt_doctorado.apply(lambda x: int(x['Finalidad transferencia'].split(' ')[-1]) , axis=1)
conicyt_doctorado.head()

,Fecha transferencia,Año transferencia,Mes transferencia,Finalidad transferencia,Respaldo transferencia,Imputación presupuestaria,Monto transferencia,Unidad monetaria,RUT,Razón social,Apellido paterno,Apellido materno,Nombres,Región de la organización,Folio
0,2010-09-21,2010,Septiembre,BECAS NACIONAL ASIG. MENSU DOCTORADO 2010 1463...,DECRETO SUPREMO 29 DEL 29/12/1988,2401221,560000.0,Pesos,NaN,NaN,AUFFARTH,II,ROBERT FREDERICK,XIII,14631527
1,2010-09-21,2010,Septiembre,BECAS NACIONAL ASIG. MENSU DOCTORADO 2010 1517...,DECRETO SUPREMO 29 DEL 29/12/1988,2401221,3360000.0,Pesos,NaN,NaN,FUENTES,INZUNZA,RODRIGO ALEJANDRO,VIII,15177658
2,2010-09-21,2010,Septiembre,BECAS NACIONAL ASIG. MENSU DOCTORADO 2010 1609...,DECRETO SUPREMO 29 DEL 29/12/1988,2401221,585000.0,Pesos,NaN,NaN,LOPEZ,ESPINOZA,PAULA CAROLINA,XIII,16094162
3,2010-09-21,2010,Septiembre,BECAS NACIONAL ASIG. MENSU DOCTORADO 2010 8376250,DECRETO SUPREMO 29 DEL 29/12/1988,2401221,697000.0,Pesos,NaN,NaN,VILLAGRÁN,VALENZUELA,MAURICIO FELIPE,XIII,8376250
4,2010-09-23,2010,Septiembre,BECAS NACIONAL ASIG. MENSU DOCTORADO 2010 7220035,DECRETO SUPREMO 29 DEL 29/12/1988,2401221,560000.0,Pesos,NaN,NaN,PEREZ,PRADO,MARIA FRANCISCA,XIII,7220035


In [119]:
conicyt_query=conicyt_doctorado.query('Folio > 21100000').sort_values('Folio')
conicyt_doctorado_nomina=conicyt_query.groupby('Folio').first().reset_index()[['Folio','Apellido paterno','Apellido materno','Nombres']]

print('Nº de Doctorados 2010: ', len(conicyt_doctorado_nomina),'\n')
conicyt_doctorado_nomina.head()

Nº de Doctorados 2010:  498 



,Folio,Apellido paterno,Apellido materno,Nombres
0,21100005,JURFEST,RIVERO,SONIA PATRICIA
1,21100007,BASCUÑÁN,RODRIGUEZ,MARÍA LUZ
2,21100010,MUÑOZ,HERNANDEZ,LORETO ANDREA
3,21100020,PEREZ DE ARCE,JERIA,MIGUEL FELIPE
4,21100023,ZAMORA,SAA,JILBERTO ANTONIO


In [122]:
conicyt_doctorado_nomina.to_csv('data/conicyt doctorado 2010.csv', index=True, index_label='index')